In [1]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from flask import Flask, jsonify
from sqlalchemy import create_engine, func
from datetime import datetime
import pandas as pd

In [2]:
engine = create_engine("sqlite:///../Resources/hawaii.sqlite")
Base = automap_base()
Base.prepare(autoload_with=engine)
# reflect the tables
Base.classes.keys()

# Save references to each table
station = Base.classes.station
measurement = Base.classes.measurement


In [3]:
app = Flask(__name__)
session = Session(engine)
session.close()

In [4]:
start_date = "2010-1-7"
end_date = "2010-4-10"

In [5]:
start_object = datetime.strptime(start_date, "%Y-%m-%d").date()
end_object = datetime.strptime(end_date, "%Y-%m-%d").date()

In [9]:
session.close()

In [6]:
print(start_object)
print(end_object)

2010-01-07
2010-04-10


In [7]:
tobs_from = session.query(
    measurement.date, 
    func.min(measurement.tobs).label('min_1'),
    func.avg(measurement.tobs).label('avg_1'),
    func.max(measurement.tobs).label('max_1')
).filter(
    measurement.date >= start_object,
    measurement.date <= end_object
).group_by(
    measurement.date
).all() 
print(tobs_from)

[('2010-01-07', 68.0, 70.42857142857143, 74.0), ('2010-01-08', 63.0, 65.57142857142857, 73.0), ('2010-01-09', 67.0, 68.57142857142857, 70.0), ('2010-01-10', 70.0, 72.71428571428571, 75.0), ('2010-01-11', 64.0, 66.71428571428571, 72.0), ('2010-01-12', 57.0, 61.57142857142857, 68.0), ('2010-01-13', 57.0, 61.5, 68.0), ('2010-01-14', 65.0, 66.42857142857143, 69.0), ('2010-01-15', 56.0, 64.83333333333333, 69.0), ('2010-01-16', 65.0, 67.33333333333333, 71.0), ('2010-01-17', 64.0, 68.0, 70.0), ('2010-01-18', 70.0, 73.0, 77.0), ('2010-01-19', 61.0, 69.42857142857143, 75.0), ('2010-01-20', 66.0, 67.85714285714286, 72.0), ('2010-01-21', 68.0, 69.5, 71.0), ('2010-01-22', 60.0, 63.5, 67.0), ('2010-01-23', 67.0, 73.83333333333333, 78.0), ('2010-01-24', 71.0, 73.16666666666667, 76.0), ('2010-01-25', 65.0, 67.14285714285714, 72.0), ('2010-01-26', 69.0, 74.71428571428571, 77.0), ('2010-01-27', 65.0, 68.71428571428571, 71.0), ('2010-01-28', 67.0, 69.83333333333333, 72.0), ('2010-01-29', 65.0, 65.8, 68.

In [12]:
tobs_from = session.query(measurement.date, 
                              func.min(measurement.tobs),\
                              func.avg(measurement.tobs),\
                                func.max(measurement.tobs)).\
                                filter(measurement.date >= start_date).\
                                filter(measurement.date <= end_date).\
                                group_by(measurement.date)

print(tobs_from)

SELECT measurement.date AS measurement_date, min(measurement.tobs) AS min_1, avg(measurement.tobs) AS avg_1, max(measurement.tobs) AS max_1 
FROM measurement 
WHERE measurement.date >= ? AND measurement.date <= ? GROUP BY measurement.date


In [8]:
tobs_from_to_list = []
for d, min, avg, max in tobs_from:
        tobs_from_data = {}
        tobs_from_data["Date"] = d
        tobs_from_data["Minimum Temperature"] = min
        tobs_from_data["Average Temperature"] = round(avg, 1)
        tobs_from_data["Maximum Temperature"] = max
        tobs_from_to_list.append(tobs_from_data)
print(tobs_from_to_list)

[{'Date': '2010-01-07', 'Minimum Temperature': 68.0, 'Average Temperature': 70.4, 'Maximum Temperature': 74.0}, {'Date': '2010-01-08', 'Minimum Temperature': 63.0, 'Average Temperature': 65.6, 'Maximum Temperature': 73.0}, {'Date': '2010-01-09', 'Minimum Temperature': 67.0, 'Average Temperature': 68.6, 'Maximum Temperature': 70.0}, {'Date': '2010-01-10', 'Minimum Temperature': 70.0, 'Average Temperature': 72.7, 'Maximum Temperature': 75.0}, {'Date': '2010-01-11', 'Minimum Temperature': 64.0, 'Average Temperature': 66.7, 'Maximum Temperature': 72.0}, {'Date': '2010-01-12', 'Minimum Temperature': 57.0, 'Average Temperature': 61.6, 'Maximum Temperature': 68.0}, {'Date': '2010-01-13', 'Minimum Temperature': 57.0, 'Average Temperature': 61.5, 'Maximum Temperature': 68.0}, {'Date': '2010-01-14', 'Minimum Temperature': 65.0, 'Average Temperature': 66.4, 'Maximum Temperature': 69.0}, {'Date': '2010-01-15', 'Minimum Temperature': 56.0, 'Average Temperature': 64.8, 'Maximum Temperature': 69.0}, 

In [26]:
blah = session.query(measurement.date, measurement.tobs, measurement.prcp)

In [27]:
tester = pd.DataFrame(blah)

In [28]:
any_nulls = tester['prcp'].isnull().values.any()
print(any_nulls)

True


In [ ]:
@app.route("/api/v1.0/start_date/end_date/<start_date>/<end_date>")
def start_end(start_date, end_date):
    
    session = Session(engine)

    tobs_from = session.query(measurement.date, 
                              func.min(measurement.tobs),\
                              func.avg(measurement.tobs),\
                                func.max(measurement.tobs)).\
                                filter(measurement.date >= start_date).\
                                filter(measurement.date <= end_date).\
                                group_by(measurement.date)


    session.close()

    print(tobs_from)

    tobs_from_to_list = []
    for d, min, avg, max in tobs_from:
        tobs_from_data = {}
        tobs_from_data["Date"] = d
        tobs_from_data["Minimum Temperature"] = min
        tobs_from_data["Average Temperature"] = round(avg, 1)
        tobs_from_data["Maximum Temperature"] = max
        tobs_from_to_list.append(tobs_from_data)

    

    return jsonify(tobs_from_to_list)

